In [1]:
import scrapy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from scrapy.selector import Selector
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta

last_x_weeeks = 4 

In [2]:
name = 'cybersecurityventures'
allowed_domains = ['www.cybersecurityventures.com']
start_urls = ['https://cybersecurityventures.com']

# set up the driver
chrome_options = Options()
# chrome_options.add_argument("--headless") # uncomment if don't want to appreciate the sight of a possessed browser
driver = webdriver.Chrome(chrome_options)
driver.get("https://cybersecurityventures.com/today/")



time.sleep(2)

In [3]:
html = [driver.page_source]

### 2. Define function to scrap each news page

In [4]:
def get_pages(html,index):
    rows = []
    for page in html:
        resp = Selector(text = page)
        #print(resp)
        results = resp.xpath(f"//*[@id='view_14']/div[3]") # result iterator
        
        
        print(results)    
        dataUriList =results.xpath(f"//*[@class='kn-list-item-container kn-list-container column is-full']")
        
        print(len(dataUriList))
        row = []
        for i,item in enumerate(dataUriList):

            id = item.xpath(".//@id").get()

            try:

                #title
                title = item.xpath(f"//*[@id='{id}']/section/div/div/div/div/div[2]/div/span/h2/span/text()").get()

                print(title)

            except:

                title = 'No Title'
            

            try:
                #date  
                date_div = item.xpath(f"//*[@id='{id}']/section/div/div/div/div/div[1]/div/span/em/span/text()").get()
                datetime_parsed =datetime.strptime(date_div, "%m/%d/%Y")
                print(datetime_parsed)
            
            except Exception as e:
                print(e)
                datetime_parsed = datetime.strptime("January 01, 2000", "%B %d, %Y")


            try:

                link = item.xpath(f"//*[@id='{id}']/section/div/div/div/div/div[3]/div/span/span/p/a/@href").get()
                print(link)
            except:
                link = 'No link'


            try:

                description = item.xpath(f"//*[@id='{id}']/section/div/div/div/div/div[3]/div/span/span/p/text()").get()
                #description = description_div.xpath(".//text()").get().strip()
                #print(description)
            
            except:
                description = 'No description'

            
            try:

                author = item.xpath(f"//*[@id='{id}']/section/div/div/div/div/div[4]/div/span/em/span/text()").get()
            except:
                author = 'No author'

            row = [title,datetime_parsed,description,link,author]
            rows.append(row)

        return rows

### 4. Order and filter news by date

In [5]:
headers = ['Title','Publish Date','Description','Url','Author']
news = get_pages(html,1)

[<Selector query="//*[@id='view_14']/div[3]" data='<div class="kn-list-content columns i...'>]
100
States of Guernsey hit by attempted cyberattack on emails
2024-07-01 00:00:00
https://www.bbc.com/news/articles/c727l04pljjo
San Diego healthcare provider's computer systems down 2 months after 'suspicious activities'
2024-07-01 00:00:00
https://www.palomarhealthmedicalgroup.org/phmg-update/
US Treasury finalizes crypto rules to prevent tax evasion
2024-07-01 00:00:00
https://www.wsj.com/finance/currencies/crypto-investors-to-see-tighter-tax-rules-starting-in-2026-fe05fdbc
Canada's Federated Co-op suffers cyber incident, systems and website down
2024-07-01 00:00:00
https://www.cbc.ca/news/canada/saskatchewan/co-op-closed-cybersecurity-1.7251150
Cybersecurity Incident Disrupts Saratoga Springs' IT System
2024-07-01 00:00:00
https://cbs6albany.com/news/local/cybersecurity-incident-disrupts-saratoga-it-system
Largest Croatian hospital under cyberattack
2024-07-01 00:00:00
https://vijesti.hrt

In [6]:
filter_date = datetime.today()-timedelta(weeks=1)


news_df = pd.DataFrame(data=news,columns=headers)
#print(news_df.head())
news_df_sorted = news_df.sort_values(by= 'Publish Date', ascending=False)
news_df_filtered = news_df_sorted[news_df_sorted['Publish Date']>=filter_date]


### 5. Build the output file to be used as chat gpt input

In [7]:
 
print(news_df_filtered)
news_df_filtered.to_csv('OutputTestFiles/csm_test.csv',sep=";")

                                                Title Publish Date  \
0   States of Guernsey hit by attempted cyberattac...   2024-07-01   
4   Cybersecurity Incident Disrupts Saratoga Sprin...   2024-07-01   
5         Largest Croatian hospital under cyberattack   2024-07-01   
1   San Diego healthcare provider's computer syste...   2024-07-01   
3   Canada's Federated Co-op suffers cyber inciden...   2024-07-01   
2   US Treasury finalizes crypto rules to prevent ...   2024-07-01   
6   TeamViewer says Russia’s ‘Cozy Bear’ hackers a...   2024-06-29   
7   Cyberattack hits Franklin County, Wash., syste...   2024-06-29   
8   Inside a violent gang’s ruthless crypto-steali...   2024-06-29   
9   Indonesia plans to expel 103 Taiwanese caught ...   2024-06-29   
10  Coinstats Security Breach Hits More Than 1,500...   2024-06-29   
11  HubSpot says it’s investigating customer accou...   2024-06-29   
17  The nation's oldest nonprofit newsroom is suin...   2024-06-28   
16  UK's Shoe Zone C